In [7]:
import numpy as np
import pandas as pd

In [8]:
class decision_node:
    def __init__(self,col_name=None,value=None,results=None,true_branch=None,false_branch=None):
        self.col_name=col_name # column name of criteria being tested
        self.value=value # vlaue necessary to get a true result
        self.results=results # dict of results for a branch, None for everything except endpoints
        self.true_branch = true_branch # true decision nodes 
        self.false_branch = false_branch # false decision nodes
        
    def classify_row(self,row):
        if self.results != None:
            return self.results.keys()[0]
        else:
            decision_col = self.col_name
            decision_value = self.value
            if isinstance(decision_value,np.int64):
                if row[decision_col] > decision_value: 
                    return self.true_branch.classify_row(row)
                else: 
                    return self.false_branch.classify_row(row)
            else:
                if row[decision_col] == decision_value:
                    return self.true_branch.classify_row(row)
                else: 
                    return self.false_branch.classify_row(row)
    
    def classify(self,df):
        predicted_list = []
        for index, row in df.iterrows(): 
            predicted_result = self.classify_row(row)
            predicted_list.append(predicted_result)
        new_df = df.copy()
        new_df['Prediction'] = predicted_list
        return new_df
    
    def print_tree(self,indent=''):
        # Is this a leaf node?
        if self.results!=None:
            print str(self.results.keys()[0])
        else:
            # Print the criteria
            print 'Variable ' + str(self.col_name)+' : >='+str(self.value)+' ? ' if isinstance(self.value, np.int64) \
                else 'Variable ' + str(self.col_name)+' : is '+str(self.value)+' ? '
            # Print the branches
            print indent+'True ->',
            self.true_branch.print_tree(indent+'  ')
            print indent+'False ->',
            self.false_branch.print_tree(indent+'  ')

In [9]:
# Divides a set on a specific column.
def divide_set(df,column,value): # rows is a df, column is string (col name)
    # Make a function that tells us if a row is in the first group 
    # (true) or the second group (false)
    split_function=None
    # for numerical values
    if isinstance(value,np.int64) or isinstance(value,np.float64):
        split_function=lambda x : x >= value
    # for nominal values
    else:
        split_function=lambda x : x == value
   
   # Divide the rows into two sets and return them
    df_true = df.loc[df[column].apply(split_function)]
    df_false = df.loc[~df[column].apply(split_function)]

    return df_true, df_false

In [10]:
from collections import defaultdict
def unique_counts(df):
    results = defaultdict(lambda: 0)
    for idnex, row in df.iterrows():
        r = row[df.columns[-1]]
        results[r]+=1
    return dict(results) 

In [11]:
# Entropy is the sum of p(x)log(p(x)) across all the different possible results
def entropy(df):
    from math import log
    log2 = lambda x:log(x)/log(2)  
    results = unique_counts(df)
    # Now calculate the entropy
    ent=0.0
    for r in results.keys():
        # current probability of class
        p=float(results[r])/df.shape[0]
        ent=ent-p*log2(p)
    return ent

In [12]:
def build_tree(df, score_function=entropy):
    if df.shape[0] == 0: return decision_node()
    current_score = score_function(df)

    best_gain = 0.0
    best_criteria = None
    best_sets = None

    print "--------NEW BRANCH-------"
    for col_name in df.columns[:-1]: #last col is result
        # find different values in this column
        col_values = set(df[col_name])

        # for each possible value, try to divide on that value
        for value in col_values:
            df_true, df_false = divide_set(df, col_name, value)

            # Information gain
            p = float(df_true.shape[0])/df.shape[0]
            gain = current_score - p*score_function(df_true) - (1-p)*score_function(df_false)
            print 
            print "Variable considered: {0} at {1}".format(col_name,value)
            print "Variable gain = ",gain, 
            if gain > best_gain and df_true.shape[0] > 0 and df_false.shape[0] > 0:
                print 
                print 
                print '  '+">>> New best criteria: {0} at {1}".format(col_name,value)
                print '  '+">>> New best gain: {0}.".format(gain)
                best_gain = gain
                best_criteria = (col_name, value)
                best_sets = (df_true, df_false)
            else: 
                print "<= actual best gain ({0})".format(best_gain)
    if best_gain > 0:
        print 
        print ">>> Best criteria for this branch: {0} at {1}.".format(best_criteria[0],best_criteria[1])
        print ">>> Best gain for this branch: {0}.".format(best_gain)
        print "\n\n"
        trueBranch = build_tree(best_sets[0])
        falseBranch = build_tree(best_sets[1])
        return decision_node(col_name=best_criteria[0], value=best_criteria[1],
                true_branch=trueBranch, false_branch=falseBranch)
    else:
        print 
        print ">>> We reach a leaf of the decision tree."
        print "\n\n"
        return decision_node(results=unique_counts(df))

Create data frame

In [13]:
Type = ['courte','courte','longue','longue','courte','longue','longue','longue','courte','courte']
Prop = ['chimique','solaire','chimique','solaire','nucleaire','chimique','chimique','solaire','nucleaire','chimique']
Astro = [6,6,3,4,6,6,4,6,3,3]
MOI = ['freinage','freinage','aerocapt.','freinage','freinage','freinage', 'aerocapt.','freinage','aerocapt.','aerocapt.']
ERV = ['terrestre','terrestre','martienne','terrestre','terrestre','terrestre','terrestre','martienne','terrestre','martienne']
EDL = ['gonflable','gonflable','rigide','gonflable','gonflable','gonflable','gonflable','gonflable','rigide','gonflable']
Eval = ['TC','TC','A','A','TC','TC','A','TC','TC','TC']

In [14]:
df = pd.DataFrame()

df['Type'] = Type
df['Prop'] = Prop
df['Astro'] = Astro
df['MOI'] = MOI
df['ERV'] = ERV
df['EDL'] = EDL
df['Eval'] = Eval

df

,Type,Prop,Astro,MOI,ERV,EDL,Eval
0,courte,chimique,6,freinage,terrestre,gonflable,TC
1,courte,solaire,6,freinage,terrestre,gonflable,TC
2,longue,chimique,3,aerocapt.,martienne,rigide,A
3,longue,solaire,4,freinage,terrestre,gonflable,A
4,courte,nucleaire,6,freinage,terrestre,gonflable,TC
5,longue,chimique,6,freinage,terrestre,gonflable,TC
6,longue,chimique,4,aerocapt.,terrestre,gonflable,A
7,longue,solaire,6,freinage,martienne,gonflable,TC
8,courte,nucleaire,3,aerocapt.,terrestre,rigide,TC
9,courte,chimique,3,aerocapt.,martienne,gonflable,TC


In [15]:
tree = build_tree(df)

--------NEW BRANCH-------

Variable considered: Type at courte
Variable gain =  0.395815602003

  >>> New best criteria: Type at courte
  >>> New best gain: 0.395815602003.

Variable considered: Type at longue
Variable gain =  0.395815602003 <= actual best gain (0.395815602003)

Variable considered: Prop at nucleaire
Variable gain =  0.117743696891 <= actual best gain (0.395815602003)

Variable considered: Prop at solaire
Variable gain =  0.0016177510177 <= actual best gain (0.395815602003)

Variable considered: Prop at chimique
Variable gain =  0.0348515545597 <= actual best gain (0.395815602003)

Variable considered: Astro at 3
Variable gain =  0.0 <= actual best gain (0.395815602003)

Variable considered: Astro at 4
Variable gain =  0.0016177510177 <= actual best gain (0.395815602003)

Variable considered: Astro at 6
Variable gain =  0.395815602003 <= actual best gain (0.395815602003)

Variable considered: MOI at aerocapt.
Variable gain =  0.0912774462417 <= actual best gain (0.3958

In [16]:
tree.print_tree()

Variable Type : is courte ? 
True -> TC
False -> Variable Astro : >=6 ? 
  True -> TC
  False -> A


In [17]:
tree.classify(df)

,Type,Prop,Astro,MOI,ERV,EDL,Eval,Prediction
0,courte,chimique,6,freinage,terrestre,gonflable,TC,TC
1,courte,solaire,6,freinage,terrestre,gonflable,TC,TC
2,longue,chimique,3,aerocapt.,martienne,rigide,A,A
3,longue,solaire,4,freinage,terrestre,gonflable,A,A
4,courte,nucleaire,6,freinage,terrestre,gonflable,TC,TC
5,longue,chimique,6,freinage,terrestre,gonflable,TC,A
6,longue,chimique,4,aerocapt.,terrestre,gonflable,A,A
7,longue,solaire,6,freinage,martienne,gonflable,TC,A
8,courte,nucleaire,3,aerocapt.,terrestre,rigide,TC,TC
9,courte,chimique,3,aerocapt.,martienne,gonflable,TC,TC
